In [2]:
import numpy as np
import open3d as o3d
import back_surface as back
import utilities as uti

## Main

In [7]:
for i in range(1,9):
    pcd = o3d.io.read_point_cloud("./Points/clean points/dwnsample_pose_"+str(i)+".ply")

    # inverse the surface
    points = np.asarray(pcd.points)
    behi = back.generateBack(points,0.05,0.1,0.08)
    bcd = uti.createPointCloud(behi) #back point cloud

    # compute normal vectors
    cam_front = np.array([0,0,0])
    cam_back = np.array([0,0,10])
    uti.computeNormal(pcd,cam_front)
    uti.computeNormal(bcd,cam_back)
    #o3d.visualization.draw_geometries([pcd,bcd])

    #merge 2 surfaces
    merge = uti.mergeTwoClouds(pcd,bcd)
    #o3d.visualization.draw_geometries([merge])

    print('run Poisson surface reconstruction')
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(merge, depth=8)
    print(mesh)
    mesh.compute_vertex_normals()
    #o3d.visualization.draw_geometries([mesh])
    #o3d.io.write_triangle_mesh('./Mesh/Poisson Mesh/normal/pose_'+str(i)+'.ply',mesh)
    mesh = uti.removeBacksurface(mesh,cam_front)
    o3d.io.write_triangle_mesh('./Mesh/Removed Backsurface/Jean normal/pose_'+str(i)+'.ply',mesh)

run Poisson surface reconstruction
geometry::TriangleMesh with 3356 points and 6678 triangles.


temp = np.asarray(pcd.points)
new_points = duplicateSurface(temp)
new_pcl = o3d.geometry.PointCloud()
new_pcl.points = o3d.utility.Vector3dVector(np.concatenate((temp,new_points)))
new_pcl.normals = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd.normals),-np.asarray(pcd.normals))))
new_pcl.paint_uniform_color([0.8, 0.8, 0.8])
#o3d.visualization.draw_geometries([new_pcl])
#o3d.io.write_point_cloud("./clean points/dupback_pose_1.ply", new_pcl)